# Exercise 3
### Computer Vision, Fall 2025
#### Name: <font color='blue'>Matias Paloranta</font>
---
**Instructions:**
- Return the answer in PDF and Jupyter Notebook formats.

- Return latest on<font color='red'> Sunday Sep. 28 at 23.50</font> via Moodle.


## Ex 3.1. CAMERA MATRIX (4 p)
Please find the 3 sets of images provided in the Moodle page of this week's exercise. The three image sets (A,B and C) contain pictures taken of a camera calibration checkerboard pattern with a square size of 3cm.

1. Write a function using built-in OpenCV methods that computes the intrinsic camera calibration matrix and distortion matrix from a given set of calibration images.

2. Apply that function to all three sets of images and observe the results. Based on the results for the intrinsic and distortion matrix, discuss what type of camera or lens was used to capture the different image sets.


In [26]:
import cv2
import numpy as np
import os
import glob
from matplotlib import pyplot as plt

np.set_printoptions(suppress=True)

W, H = 3024, 4032 # image width and height in pixels

def compute_fov(matrix):
    fx, fy = matrix[0, 0], matrix[1, 1]
    fov_x = np.degrees(2*np.arctan(W/(2*fx)))
    fov_y = np.degrees(2*np.arctan(H/(2*fy)))
    return fov_x, fov_y

def print_principal_point(matrix):
    p_x, p_y = matrix[0, 2], matrix[1, 2] # principal point
    print(f"Ṕrincipal point: ({p_x:.0f}, {p_y:.0f})")
    print(f"Image center: ({W/2}, {H/2})")

def calibrate(img_path):
    matrix = None
    distortion = None
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    points_row, points_col = (6, 8) # number of inner corners is number of squares - 1 in any given dimension
    square_size = 0.03 # in meters
    objp = np.zeros((points_col*points_row, 3), np.float32)
    objp[:,:2] = np.mgrid[0:points_row, 0:points_col].T.reshape(-1,2) * square_size
 
    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane.

    images = glob.glob("{}/*.jpeg".format(img_path))

    for filename in images:
        image = cv2.imread(filename)
        grayColor = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
         
        # Find the chess board corners
        ret, corners = cv2.findChessboardCornersSB(grayColor, (points_row, points_col), None)
    
        # If found, add object points, image points (after refining them)
        if ret == True:
            objpoints.append(objp)
    
            corners2 = cv2.cornerSubPix(grayColor, corners, (11,11), (-1,-1), criteria)
            imgpoints.append(corners2)
            
    #         cv2.drawChessboardCorners(image, (points_row, points_col), corners2, ret)
    #         cv2.namedWindow("img", cv2.WINDOW_NORMAL)
    #         cv2.imshow("img", image)
    #         cv2.waitKey(0)
    # cv2.destroyAllWindows()
        
    # fix higher order terms to avoid overfitting
    flags = cv2.CALIB_FIX_K3 | cv2.CALIB_FIX_K4 | cv2.CALIB_FIX_K5
    ret, matrix, distortion, rvecs, tvecs = cv2.calibrateCamera(
        objpoints, imgpoints, grayColor.shape[::-1], None, None, flags=flags
    )
    
    mean_error = 0
    for i in range(len(objpoints)):
        imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], matrix, distortion)
        error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
        mean_error += error
    
    print( "total error: {}".format(mean_error/len(objpoints)) )
    
    return matrix, distortion

print("Images A:")
matrix, distortion = calibrate("images_A")
print(matrix)
print(distortion)
fov_x, fov_y = compute_fov(matrix)
print(f"Fov (x, y): {fov_x:.1f}, {fov_y:.1f} degrees")
print_principal_point(matrix)
print()

print("Images B:")
matrix, distortion = calibrate("images_B")
print(matrix)
print(distortion)
fov_x, fov_y = compute_fov(matrix)
print(f"Fov (x, y): {fov_x:.1f}, {fov_y:.1f} degrees")
print_principal_point(matrix)
print()

print("Images C:")
matrix, distortion = calibrate("images_C")
print(matrix)
print(distortion)
fov_x, fov_y = compute_fov(matrix)
print(f"Fov (x, y): {fov_x:.1f}, {fov_y:.1f} degrees")
print_principal_point(matrix)

Images A:
total error: 0.15474618043978428
[[3553.30638915    0.         1623.55454607]
 [   0.         3552.90679709 1860.43967695]
 [   0.            0.            1.        ]]
[[ 0.12107222 -0.08137645 -0.01189253  0.01028686  0.        ]]
Fov (x, y): 46.1, 59.1 degrees
Ṕrincipal point: (1624, 1860)
Image center: (1512.0, 2016.0)

Images B:
total error: 0.21089247090030022
[[1706.66147505    0.         1477.37254157]
 [   0.         1703.3379684  1989.93428911]
 [   0.            0.            1.        ]]
[[-0.00245321  0.01280024 -0.00171085 -0.00364829  0.        ]]
Fov (x, y): 83.1, 99.6 degrees
Ṕrincipal point: (1477, 1990)
Image center: (1512.0, 2016.0)

Images C:
total error: 0.24184582728248252
[[7802.99246987    0.          889.01807348]
 [   0.         7751.04024232 1573.22511339]
 [   0.            0.            1.        ]]
[[ 0.20523726  0.1403325  -0.03772315 -0.03353585  0.        ]]
Fov (x, y): 21.9, 29.2 degrees
Ṕrincipal point: (889, 1573)
Image center: (1512.0, 20

### Images A:

Based on FOV a quite normal lens, some barrel distortion radially.

### Images B:

Wide lens, well calibrated.

### Images C:

Pincushion distortion and narrow FOV indicates telephoto or zoom, poorly calibrated due to large offset of the principal point from image center.

## Ex 3.2. HOUGH TRANSFORM (4 p)
1. Ask ChatGPT to explain the main points about Hough Transform using the key word "Hough Transform". Analyse the answer by reflecting it to the course material related to Hough Transform: Were all main points in the course material addressed? if no, please shortly conclude the answers of ChatGPT and our course materials regarding the Hough Transform, and list the differences of answers between our course material and ChatGPT.
2. We provided you example images from the Berkeley Segmentation Dataset. Implement the Hough Transform (the most basic version) for line detection by yourself (You may only use the openCV function to check if your solution is correct). Instead of using the default parameters, customize the parameters (e.g., resolution of the parameter space, threshold values for line detection, minimum line length, maximum line gap) to optimize edge detection for the given set of images. Report your observations: which parameters influenced which behaviour in the output? Also report which parameter configuration resembles closest an object segmentation in the test images.


In [ ]:
#2)
import glob
import numpy as np
import matplotlib.pyplot as plt
import cv2

def apply_hough_transform(image):
    ...


images = glob.glob("segmentation_images/*.jpg")

for filename in images:
    image = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    ...